In [ ]:
import pandas as pd
import numpy as np
from igraph import *
import igraph

## Load graph

In [ ]:
data = pd.read_csv('club_members.csv')

In [ ]:
data.rename(columns={'Unnamed: 0': 'name'}, inplace=True)

In [ ]:
data.head()

In [ ]:
a = data.iloc[:,1:].as_matrix()

In [ ]:
a.shape

In [ ]:
persons = np.dot(a, a.T)

In [ ]:
persons.shape

In [ ]:
persons

In [ ]:
np.fill_diagonal(persons, 0)

In [ ]:
persons

In [ ]:
g = Graph.Adjacency(persons.tolist(), igraph.ADJ_UNDIRECTED)

In [ ]:
g

In [ ]:
g.summary()

In [ ]:
g.vs['name'] = data.iloc[:,0].tolist()

In [ ]:
g.layout_fruchterman_reingold()
plot(g, 
#      vertex_label = g.vs.get_attribute_values('name'),
     edge_curved=0.3,
     edge_color='#aaaaaa',
     vertex_color="#ff9595",
     vertex_label_size = 8,
#      vertex_shape='hidden'
)

## "Important" persons

In [ ]:
def get_top(g, metric, n=10):
    return sorted(zip(getattr(g, metric)(), g.vs['name']), reverse=True)[:n]

In [ ]:
get_top(g, 'degree')

In [ ]:
get_top(g, 'closeness')

In [ ]:
important_persons =  pd.DataFrame({
    'degree': [name for _, name in get_top(g, 'degree')],
    'closeness': [name for _, name in get_top(g, 'closeness')],
    'betweenness': [name for _, name in get_top(g, 'betweenness')],
    'eigenvector_centrality': [name for _, name in get_top(g, 'eigenvector_centrality')],
    'pagerank': [name for _, name in get_top(g, 'pagerank')],
})

In [ ]:
important_persons

How is **Paul Revere?** - https://en.wikipedia.org/wiki/Paul_Revere

## Clusters

In [ ]:
multilevel_communities = g.community_multilevel()

In [ ]:
multilevel_communities.sizes()

In [ ]:
multilevel_communities.modularity

In [ ]:
walktrap = g.community_walktrap(steps=10)

In [ ]:
walktrap_communities = walktrap.as_clustering(n=10)

In [ ]:
walktrap_communities.sizes()

In [ ]:
g.vs.set_attribute_values('multilevel_communities',multilevel_communities.membership)
g.vs.set_attribute_values('walktrap_communities',walktrap_communities.membership)

In [ ]:
g.write('rebels.graphml')